In [2]:
import fipy
from toolz.curried import curry, fcompose

ModuleNotFoundError: No module named 'toolz'

In [3]:
def get_mesh(params):
    return fipy.Grid1D(nx=params['nx'], Lx=params['delta'])


def mask_right(mesh):
    mask = fipy.CellVariable(mesh=mesh)
    mask.setValue(1, where=mesh.x > (mesh.Lx - mesh.dx / 2)
    return mask

                  
def get_vars(params):
    return pipe(
        get_mesh(params),
        lambda x: dict(
            sup=fipy.CellVariable(x.mesh, name='sup', value=params['sup_ini'])),
            theta=dict('new' : 0.0, 'old' : 0.0),
            flux=fipy.CellVariable(x.mesh, value=0.)
        )
    )


def get_inf(var):
    return np.array(var[-1])


@curry
def update_theta(params, var, vars_):
    def num():
        return (var['old'] + params['dt'] * params['k_plus'] * get_inf(vars_['supp']))
    def denom():
        return params['k_plus'] * get_inf(vars_['supp']) + params['k_minus'] * params['vel']
    return dict(new=num() / (1 + params['dt'] * denum()),
                old=var[1])


@curry
def update_theta_old(params, var, vars_):
    return (var[0], var[0])
                   

@curry
def update_flux(params, var, vars_):
    var.setValue(-params['gamms'] * params['k_plus'] * (1 - vars_['theta']['theta'])) 
    return var
               

@curry
def update_flux_old(params, var, vars_):
    return update_flux(params, var, vars_)

    
@curry
def update_sup(eqn, params, var, vars_):
    eqn.sweep(var, dt=params['dt'])
    return var


@curry
def update_sup_old(params, var, vars_):
    var.update_old()
    return var


def get_eqn(flux, params):
    return TransientTerm() == DiffusionTerm(params['diff_sup']) + \
        fipy.ImplicitSourceTerm(flux * get_mask(flux.mesh) / flux.mesh.dx)


def get_system(params):
    return pipe(
        params,
        get_vars,
        lambda x: ((x['flux'], update_flux(params), update_flux_old(params)),
                   (x['theta'], update_theta(params), update_theta_old(params)),
                   (x['sup'], update_sup(get_eqn(x['flux']params), update_sup_old(params)))
    )



SyntaxError: invalid syntax (<ipython-input-3-b00fc33731b3>, line 8)

# Solving the S-NDR Equations in 1D

The equations are,

$$ \partial_t C_i = D_i \partial_x^2 C_i $$

for $i=\text{Cu},\;\text{sup}$ and

$$ \partial_t \theta = k^+ c_{\text{sup}} \left( 1 - \theta \right) - k^- \theta v $$

where

$$ v = j_0 \left( 1 - \theta \right) + j_1 \theta $$

The current is given by,

$$ j_k = j_k^{\alpha} \left[ \exp \left( \bar{\alpha} \eta \right) - \exp \left( \left(1 - \bar{\alpha}\right) \eta \right) \right] \frac{ c_{\text{Cu}} } { c_{\text{Cu}}^{\infty} }$$

for $k=0, 1$ and $\bar{\alpha} = \frac{\alpha n}{R T}$.

## Initial Conditions

$\theta = 0$, $c_{\text{Cu}} = 0$ and $c_{\text{suppressor}} = 0$

## Boundary Conditions

$c_{\text{Cu}} = c_{\text{Cu}}^{\infty}$ at the boundary layer

$c_{\text{sup}} = c_{\text{sup}}^{\infty}$ at the boundary layer

$-D_{\text{Cu}} \partial_x c_{\text{Cu}} = \frac{v}{\Omega}$ at the electrode

$-D_{\text{sup}} \partial_x c_{\text{sup}} = c_{\text{sup}} k^+ \Gamma \left( 1 - \theta \right)$ at the electrode

## Parameters



|           Symbol           |                 Value (a) |
|:--------------------------:|--------------------------:|
|       $D_\text{supp}$      |   $9.2\;\times\;10^{-11}$ |
|       $D_\text{Cu}$        | $2.65\;\times\;10^{-10}$  |
| $c_{\text{Cu}}^{\infty}$   |                   240     |
| $c_{\text{sup}}^{\infty}$  | 0.01                      |
|       $\delta$             |   $1\;\times\;10^{-6}$    |
|  $ \Gamma  $                 |   $2.5\;\times\;10^{-7}$  |
| $k^+$                      |          2300             |
| $k^-$                      |   $3.79\;\times\;10^{7}$  |
| $j_0^{\alpha}$             |          20               |
| $j_1^{\alpha}$             |   $1\;\times\;10^{-3}$    |
| $\alpha$                   |   0.4                     |
| $n$                        |   2                       |
| $\Omega$                   |   $7.2\;\times\;10^{-6}$  |
| $F$                        |                           |
| $V_0$ (initial potential)  | 0                         |
| $V_s$ (switch)             | -0.325                    |
| $V_mmp$ (ramp rate)        | 0.01                      |
| Final time $t_f$           | 65                        |